In [1]:
from __future__ import print_function, division
%matplotlib inline
import statsmodels.formula.api as smf  
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import sys
from scipy import stats
from fiona.crs import from_epsg
import shapely
#from pysal.cg.shapes import Polygon

In [2]:
df = pd.read_csv("five_variables_zipcode.csv", index_col=0)
df['zip_code'] = df['zip_code'].astype(str)
df.head()

,zip_code,traffic,GHG,boiler,construction,pm,traffic_square
0,10034,2.890238,-0.540985,1.072069,-0.763984,0.184157,3.934167
1,10017,2.841724,2.294960,0.563143,1.477751,2.498788,3.837417
2,10009,2.613907,0.005641,-0.575281,0.387327,0.888610,3.395591
3,10475,2.592991,0.149569,-0.689843,-0.835931,-0.117751,3.356060
4,10011,2.396429,0.032177,1.485571,3.104135,2.196880,2.993048


In [3]:
os.system("curl -O http://catalog.civicdashboards.com/dataset/11fd957a-8885-42ef-aa49-5c879ec93fac/resource/28377e88-8a50-428f-807c-40ba1f09159b/download/nyc-zip-code-tabulation-areas-polygons.geojson")
os.system("mv " + "nyc-zip-code-tabulation-areas-polygons.geojson " + os.getenv("PUIDATA"))
nyc = gpd.read_file(os.getenv("PUIDATA") + "/" + 'nyc-zip-code-tabulation-areas-polygons.geojson')

In [4]:
nyc['zip_code'] = nyc.postalCode.astype(str)
nyc.drop(['postalCode'], axis=1, inplace=True)

In [5]:
mapData = pd.merge(nyc, df, on='zip_code').drop_duplicates('zip_code')
mapData = gpd.GeoDataFrame(mapData)
mapData.head()

,OBJECTID,PO_NAME,STATE,borough,ST_FIPS,CTY_FIPS,BLDGpostal,@id,longitude,latitude,geometry,zip_code,traffic,GHG,boiler,construction,pm,traffic_square
0,1,Jackson Heights,NY,Queens,36,081,0,http://nyc.pediacities.com/Resource/PostalCode...,-73.883573,40.751662,POLYGON ((-73.86942457284177 40.74915687096788...,11372,-0.467862,-0.649442,0.064432,-0.434799,-0.017115,-0.555573
1,2,Glen Oaks,NY,Queens,36,081,0,http://nyc.pediacities.com/Resource/PostalCode...,-73.711608,40.745366,POLYGON ((-73.71132911125308 40.74947450816085...,11004,-0.311005,0.820258,-0.716272,-0.920499,-1.325385,-0.445581
2,5,Fresh Meadows,NY,Queens,36,081,0,http://nyc.pediacities.com/Resource/PostalCode...,-73.794626,40.739903,"POLYGON ((-73.81088634744756 40.7271718757592,...",11365,-1.576896,-0.723556,-0.196485,-0.422458,-1.023476,-1.054431
3,6,Elmhurst,NY,Queens,36,081,0,http://nyc.pediacities.com/Resource/PostalCode...,-73.878423,40.738851,POLYGON ((-73.88722098467159 40.72752618766135...,11373,-0.454392,-0.428756,-0.171503,-0.103707,-0.017115,-0.546510
4,8,Forest Hills,NY,Queens,36,081,0,http://nyc.pediacities.com/Resource/PostalCode...,-73.846278,40.720682,POLYGON ((-73.85644802697581 40.73625356975942...,11375,-0.340464,-0.254028,0.253172,-0.250569,-0.721568,-0.466984


In [6]:
mapData.dtypes

OBJECTID            int64
PO_NAME            object
STATE              object
borough            object
ST_FIPS            object
CTY_FIPS           object
BLDGpostal          int64
@id                object
longitude         float64
latitude          float64
geometry           object
zip_code           object
traffic           float64
GHG               float64
boiler            float64
construction      float64
pm                float64
traffic_square    float64
dtype: object

In [7]:
mapData.crs = from_epsg(2263)

In [8]:
mapData.to_file("ny_air_2015.shp")